In [11]:
import pandas as pd
import re
import openpyxl
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import Label
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
from bokeh.io import output_notebook
from bokeh.models import NumeralTickFormatter
import jdatetime
import pandas as pd
import random
import seaborn as sns


## Retriving Excel

In [12]:
df = pd.concat(pd.read_excel("/Users/mohammad/Dropbox/XAV coffee works/XAV Revenue/Excels/1403/بهار 1403/03 - خرداد/سفارشات کالا خرداد 1403.xlsx", sheet_name=None), ignore_index=True)
df2 = pd.concat(pd.read_excel("/Users/mohammad/Dropbox/XAV coffee works/XAV Revenue/Excels/1403/بهار 1403/02 - اردیبهشت/سفارشات کالا اردیبهشت 1403.xlsx",sheet_name=None),ignore_index=True)
df= pd.concat([df, df2],ignore_index=True)



df["ردیف"] = df["ردیف"].fillna(method="ffill")
df.dropna(subset=["نام محصول"], inplace=True)
df = df.set_index("ردیف")
df = df.reset_index()
for i in df.index.to_list()[:-1]:
    row  = df.loc[i,"ردیف"]
    next_row = df.loc[i+1,"ردیف"]
    if next_row == row:
        df.loc[i+1,"تاریخ سفارش"] = df.loc[i,"تاریخ سفارش"]
        df.loc[i+1,"نام"] = df.loc[i,"نام"]
        df.loc[i+1,"نام خانوادگی"] = df.loc[i,"نام خانوادگی"]
        df.loc[i+1,"نام کافه"] = df.loc[i,"نام کافه"]
        df.loc[i+1,"شهر"] = df.loc[i,"شهر"]
        df.loc[i+1,"کد پیگیری"] = df.loc[i,"کد پیگیری"]



def convert_date(string):
    splited = string.split(" ")
    day_string = splited[0]
    day = splited[-3]
    month_string = splited[-2]
    month = None
    if month_string == "فروردین":
        month = 1
    elif month_string == "اردیبهشت":
        month = 2
    elif month_string == "خرداد":
        month = 3
    elif month_string == "تیر":
        month = 4
    elif month_string == "مرداد":
        month = 5
    elif month_string == "شهریور":
        month = 6
    elif month_string == "مهر":
        month = 7

    elif month_string == "آبان":
        month = 8

    elif month_string == "آذر":
        month = 9

    elif month_string == "دی":
        month = 10

    elif month_string == "بهمن":
        month = 11

    elif month_string == "اسفند":
        month = 12
    year = splited[-1]

    convert_date = f"{year}/{month}/{day}"

    return convert_date

df.dropna(subset=["تاریخ سفارش"], inplace=True)
df.dropna(subset=["تعداد"], inplace=True)

df["تاریخ سفارش"]  = df["تاریخ سفارش"].apply(convert_date)

for i in df.index.to_list():
    try:
        count = int(df.loc[i,"تعداد"])
        df.loc[i,"واحد فرعی"] = df.loc[i,"واحد فرعی"] * count
    except:
        continue

start = str(input("start date of report :  "))
end = str(input("end date of report :  "))

start_day = int(start.split("/")[-1])
start_month = int(start.split("/")[-2])
year = int(start.split("/")[0])

end_day = int(end.split("/")[-1])
end_month = int(end.split("/")[-2])

date_list = []
if start_day <= end_day :
    # if both dates are in same month
    for i in range(start_day,end_day+1):
        date = str(year) + "/" + str(start_month) + "/" + str(i)
        date_list.append(date)

else:
    # first month dates
    for i in range(start_day,32):
        date =  str(year) + "/" + str(start_month) + "/" + str(i)
        date_list.append(date)
    
    # second month dates
    for i in range(1,end_day+1):
        date =  str(year) + "/" + str(end_month) + "/" + str(i)
        date_list.append(date)           
    
date_list

span_df =  df[df["تاریخ سفارش"].isin(date_list)]


# Deleting gifts and office use
rows_to_delete = []
for i in span_df.index.to_list():
    code = span_df.loc[i,"کد پیگیری"]
    name = span_df.loc[i,"نام"]
    if str(code)[:4] == "هدیه" or str(code)[:4] == "مصرف" or str(name)[:4] == "هدیه" or str(name)[:4] == "مصرف":
        rows_to_delete.append(i)

gifts_df = span_df.loc[rows_to_delete]
span_df = span_df.drop(rows_to_delete)


# whole sale 
total_sale = sum(span_df["واحد فرعی"].to_list())/1000



# Each Line Sale
def change_name(string):
    name = string.split("|")[-1]
    return name

span_df["نام محصول"] = span_df["نام محصول"].apply(change_name)

v_dict = {}
a_dict = {}
x_dict = {}
for i in list(span_df["نام محصول"].value_counts().keys()):
    a = span_df[span_df["نام محصول"] == i]
    kg =  round(sum(a["واحد فرعی"].to_list())/1000,3)
    code = str(span_df[span_df["نام محصول"] == i ]["کد محصول"].to_list()[0])[0]
    serie = None
    if code == "5":
        serie = "V"
        v_dict[i] = kg
    elif code == "3":
        serie = "A"
        a_dict[i] = kg
    
    elif code == "1":
        serie = "X"
        x_dict[i] = kg

# Each Cafe Buy
cafe_list = set(span_df["نام کافه"].to_list())
cafe_dict = {}
for i in cafe_list:
    buy = round(sum(span_df[span_df["نام کافه"]==i]["واحد فرعی"].to_list())/1000,3)
    cafe_dict[i] = {"kg": buy}

# Finding top 10 cafe
top_cafe = sorted(cafe_dict, key=lambda k: cafe_dict[k]['kg'], reverse=True)[:11]
top_cafe_dict = {}
for i in top_cafe:
    top_cafe_dict[i] = cafe_dict[i]["kg"]


# Each City Buy
city_list = set(span_df["شهر"].to_list())
city_dict = {}
for i in city_list:
    buy = round(sum(span_df[span_df["شهر"]==i]["واحد فرعی"].to_list())/1000,3)
    city_dict[i] = {"kg": buy}

# Finding top 10 cafe
top_city = sorted(city_dict, key=lambda k: city_dict[k]['kg'], reverse=True)[:11]
top_city_dict = {}
for i in top_city:
    top_city_dict[i] = city_dict[i]["kg"]


# plotting Total sale based on the series
total_sale = total_sale
v_series_sale = sum(v_dict.values())
a_series_sale = sum(a_dict.values())
x_series_sale = sum(x_dict.values())


plot_dict = {"X Series" : x_series_sale , "A Series":  a_series_sale, "V Series": v_series_sale, "Total": total_sale}

def each_line(dict1, string):  

    x = list(dict1.keys())
    y = list(dict1.values())

    # Create a ColumnDataSource
    colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(x))]
    source = ColumnDataSource(data=dict(x=x, y=y, color=colors))

    # Create the figure
    p = figure(x_range=x, height=600, width=1000 ,title=f"Each {string} Sale This Week",
            toolbar_location=None, tools="")

    # Render the bars
    p.vbar(x='x', top='y', width=0.5, source=source, color="color")
    

    # Customize the plot
    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    p.xaxis.major_label_orientation = 45  # Adjust the angle of the x-axis labels
    if len(x) == 1:
        x_offset = 350
    elif len(x) == 2:
        x_offset = 220
    elif len(x) == 3:
        x_offset = 70
    elif len(x) == 4:
        x_offset = 110
    elif len(x) == 5:
        x_offset = 85
    elif len(x) ==  6 :
        x_offset = 70
    elif len(x) == 7:
        x_offset = 40
    elif len(x) == 8 :
        x_offset = 55
    elif len(x) == 9:
        x_offset= 40
    elif len(x)>10:
        x_offset=40

    for i, value in enumerate(y):
        label = Label(x=i, y=value, text=str(round(value, 2)), text_baseline='middle', text_align='center', text_font_size="8pt", x_offset=x_offset, y_offset=10)
        p.add_layout(label)

    # Show the plot
    output_notebook()
    show(p)

each_line(plot_dict,"Series")

# plotting Each Line of each series sale
each_line(v_dict, "V Series")
each_line(a_dict, "A Series")
each_line(x_dict, "X Series")
each_line(top_cafe_dict, "Cafe")
each_line(top_city_dict, "City")



/Users/mohammad/Documents/GitHub/XAV/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/mohammad/Documents/GitHub/XAV/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/Users/mohammad/Documents/GitHub/XAV/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/mohammad/Documents/GitHub/XAV/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/Users/mohammad/Documents/GitHub/XAV/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/mohammad/Documents/GitHub/XA

TypeError: Object of type Undefined is not JSON serializable